# Scraping the top repos for Topics on GitHub

TODO:
- Introduction to web scraping.
- Introduction to Github (contains millions of open source repositories) and the problem statement.
- Tools we are using 
 - Python
 - requests
 - BeautifulSoup4
 - Pandas


Outline:
Here're the steps we will follow:
- We're going to scrape : https://github.com/topics
- We'll get a list of topics. For each topic, we will get topic title, topic page URL and topic description.
- For each topic, we'll get top 25 repositories in the topic from topic page.
- For each repository, we'll grab the repo name, username, stars and repo URL
- For each topic, we'll create a csv file in the following format:

```
Repo Name,Username,Stars ,Repo URL
three.js,mrdoob,69700,https://github.com/mrdoob/three.js
libgdx,libgdx,18300,https://github.com/libgdx/libgdx
```